In [ ]:
from biotransformers import BioTransformers
BioTransformers.list_backend()

In [ ]:
import torch
from Bio import SeqIO
import numpy as np
import pandas as pd

# sequences = ["MKTVRQE"]esm1_t6_43M_UR50S,esm1b_t33_650M_UR50S
from biotransformers import BioTransformers
bio_trans = BioTransformers(backend="esm1_t6_43M_UR50S")
seqList = []  
idList = []
for seq_record in SeqIO.parse('./zn15.fasta', "fasta"):
    seq = str(seq_record.seq).upper()
    id = str(seq_record.id)
    if len(seq) < 15:
        add_aa = 'A' * (15 - len(seq))
        seq = seq + add_aa   
    seqList.append(str(seq))
    idList.append(str(id))

embedds,labels = [],[]
# train_dic = {"ID": idList, "sequence": seqList,"embedd":[], "label":[]}
seq2embedding = {} 
for seq1 in seqList:
    embedding = bio_trans.compute_embeddings([seq1], pool_mode=('full'),batch_size=1,n_seqs_msa=6,silent = False)
    embedding = embedding['full']
    embedding = np.array(embedding,dtype=np.float32)
    embedding = np.squeeze(embedding, axis=0) 
    embedds.append(embedding)
    labels.append(1)
    

mg_dic = {"ID": idList, "sequence": seqList,"embedd":embedds, "label":labels}

In [3]:
len(mg_dic['ID'])

324

In [ ]:
import pickle
f_save = open('zn_dic_650M.pkl', 'wb')
pickle.dump(mg_dic, f_save)
f_save.close()

In [14]:
import pickle
import pandas as pd
import numpy as np
file = open('./zn_dic_85M.pkl', mode='rb')
fneg_dic = pickle.load(file)
file.close()


In [15]:
file = open('./zn_emb.pkl', mode='rb')
fneg_embed = pickle.load(file)
file.close()

In [16]:
import numpy as np
list1 = []
for i in range(len(fneg_embed)):
    con_embedd =np.concatenate((fneg_dic['embedd'][i], fneg_embed[i]), axis=1)
    list1.append(con_embedd) 

In [20]:
fneg_dic['embedd'] = fneg_embed

In [17]:
fneg_dic['embedd'] = list1

In [23]:
fneg_dic['embedd'][4].shape

(15, 1024)

In [24]:
f_save = open('zn_dic_BFD.pkl', 'wb')
pickle.dump(fneg_dic, f_save)
f_save.close()

In [ ]:
from biotransformers import BioTransformers
import torch
from Bio import SeqIO
import numpy as np
import pandas as pd

bio_trans = BioTransformers(backend="esm1b_t33_650M_UR50S")

seqList = []  
idList = []
for seq_record in SeqIO.parse('./negative15.fasta', "fasta"):
    seq = str(seq_record.seq).upper()
    id = str(seq_record.id)
    if len(seq) < 15:
        add_aa = 'A' * (15 - len(seq))
        seq = seq + add_aa 
    seqList.append(str(seq))
    idList.append(str(id))

embedds,labels = [],[]
neg_dic = {"ID": idList, "sequence": seqList,"embedd":[], "label":[]}
seq2embedding = {} 
for seq1 in seqList:
    embedding = bio_trans.compute_embeddings([seq1], pool_mode=('full'),batch_size=8)
    embedding = embedding['full']
    embedding = np.array(embedding,dtype=np.float32)#转为np，shape查看维度
    embedding = np.squeeze(embedding, axis=0) 
    embedds.append(embedding)
    labels.append(0)
    
    # embedding = torch.FloatTensor(embedding)
    # embedding = embedding.squeeze(0)
    # seq2embedding[seq1] = embedding

neg_dic = {"ID": idList, "sequence": seqList,"embedd":embedds, "label":labels}

In [5]:
import pickle
import pandas as pd
file = open('./neg_dic_85M.pkl', mode='rb')
neg85_dic = pickle.load(file)
file.close()


In [6]:
import numpy as np
list1 = []
for i in range(len(neg85_dic['embedd'])):
    con_embedd =np.concatenate((neg85_dic['embedd'][i], neg_dic['embedd'][i]), axis=1)
    list1.append(con_embedd) 

In [7]:
neg_dic['embedd'] = list1

In [11]:
neg_dic['embedd'][100].shape

(15, 2048)